In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import zipfile

DATA_IN_PATH = '../input/word2vec-nlp-tutorial/'

file_list = ['labeledTrainData.tsv.zip','unlabeledTrainData.tsv.zip','testData.tsv.zip']

for file in file_list:
    zipRef = zipfile.ZipFile(DATA_IN_PATH+file,'r')
    zipRef.extractall(DATA_IN_PATH)
    zipRef.close()


In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
import pandas as pd
train_data = pd.read_csv('../input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip',  delimiter="\t", quoting=3)
test_data = pd.read_csv('../input/word2vec-nlp-tutorial/testData.tsv.zip', delimiter = "\t", quoting= 3 )
submission1 = pd.read_csv("../input/word2vec-nlp-tutorial/sampleSubmission.csv")

In [ ]:
train_data.head()

In [ ]:
len(train_data)

In [ ]:
train_length = train_data['review'].apply(len)
train_length.head(10)

In [ ]:
!pip install wordcloud

In [ ]:
from wordcloud import WordCloud
cloud = WordCloud(width=800, height=600).generate(" ".join(train_data['review']))
plt.figure(figsize=(20,15))
plt.imshow(cloud)
plt.axis("off")

In [ ]:
!pip install bs4
import re
import pandas
import numpy
import json
from bs4 import BeautifulSoup as bs
from nltk.corpus import stopwords
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer



In [ ]:
print(train_data['review'][0])

In [ ]:
review = train_data['review'][0]
#HTML tag 제거하기 BeautifulSoup를 이용하여
review_text = BeautifulSoup(review,"html5lib").get_text()
#영어 글자를 제외한 나머지 공백으로 바꾸기
review_text = re.sub("[^a-zA-Z]"," ",review_text)

print(review_text)


In [ ]:
#불용어(stopword) 삭제하기
#NLTK의 불용어 사전 이용하기
#NLTK 불용어사전은 소문자로 되어있으니 소문자로 바꾸기
#불용어 set만들기
stop_words = set(stopwords.words('english'))


review_text = review_text.lower()
words = review_text.split()
words = [w for w in words if not w in stop_words]

print(words)


In [ ]:
#문자열이 단어 리스트로 변환됨
#모델에 적용시키기 위해서 다시 하나의 문자열로 합치기

clean_review = ' '.join(words)
print(clean_review)

In [ ]:
def preprocessing(review,remove_stopwords=False):
    #불용어 제거는 옵션
    
    #HTML 태그 제거하기
    review_text = BeautifulSoup(review,"html5lib").get_text()
    
    #영어가 아닌 문자 정규식을 활용하여 공백으로 바꾸기
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    
    #대문자를 소문자로 바꾸고 공백 단위로 텍스트를 나눠서 리스트로 만든다.
    
    words = review_text.lower().split
    
    #불용어 제거
    if remove_stopwords:
        #불용어사전 불러오기
        stops = set(stopwords.words("english")) 
        
        #불용어를 제거한 리스트 생성
        words = [w for w in words if not w in stops]
        
        #단어 리스트를 하나의 글로 합치기
        clean_review = ' '.join(words)
        
    #불용어를 제거하고 싶지 않음
    else:
        clean_review = ' '.join(words)
        
    return clean_review
    

In [ ]:
train_data['review'].head()

In [ ]:
clean_train_reviews = []
for review in train_data['review']:
    clean_train_reviews.append(preprocessing(review,remove_stopwords=True))

clean_train_reviews[0]

In [ ]:
def preprocessing( review, remove_stopwords = False ): 
    # 불용어 제거는 옵션으로 선택 가능하다.
    
    # 1. HTML 태그 제거
    review_text = BeautifulSoup(review, "html5lib").get_text()	

    # 2. 영어가 아닌 특수문자들을 공백(" ")으로 바꾸기
    review_text = re.sub("[^a-zA-Z]", " ", review_text)

    # 3. 대문자들을 소문자로 바꾸고 공백단위로 텍스트들 나눠서 리스트로 만든다.
    words = review_text.lower().split()

    if remove_stopwords: 
        # 4. 불용어들을 제거
    
        #영어에 관련된 불용어 불러오기
        stops = set(stopwords.words("english"))
        # 불용어가 아닌 단어들로 이루어진 새로운 리스트 생성
        words = [w for w in words if not w in stops]
        # 5. 단어 리스트를 공백을 넣어서 하나의 글로 합친다.	
        clean_review = ' '.join(words)

    else: # 불용어 제거하지 않을 때
        clean_review = ' '.join(words)

    return clean_review

In [ ]:
clean_train_reviews = []
for review in train_data['review']:
    clean_train_reviews.append(preprocessing(review, remove_stopwords = True))

# 전처리한 데이터 출력
clean_train_reviews[0]

In [ ]:
clean_train_df = pd.DataFrame({'review':clean_train_reviews,'sentiment':train_data['sentiment']})

In [ ]:
#남은 전처리 과정
#입력값이 텍스트가 아닌 각 단어의 인덱스로 되어있어야함
#길이는 동일해야 함

tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_train_reviews)
text_sequences = tokenizer.texts_to_sequences(clean_train_reviews)

print(text_sequences[0])

In [ ]:
word_vocab = tokenizer.word_index
print(word_vocab)

In [ ]:
print(len(word_vocab))

In [ ]:

data_configs = {}

data_configs['vocab'] = word_vocab
data_configs['vocab_size'] = len(word_vocab)

In [ ]:
#0으로 패딩

MAX_SEQUENCE_LENGTH = 174

train_inputs = pad_sequences(text_sequences,maxlen=MAX_SEQUENCE_LENGTH,padding='post')

print(train_inputs.shape)

In [ ]:
train_labels = np.array(train_data['sentiment'])
print('Shape of label tensor:', train_labels.shape)

In [ ]:
TRAIN_INPUT_DATA = 'train_input.npy'
TRAIN_LABEL_DATA = 'train_label.npy'
TRAIN_CLEAN_DATA = 'train_clean.csv'
DATA_CONFIGS = 'data_configs.json'



In [ ]:
# 전처리 된 데이터를 넘파이 형태로 저장
np.save(open(TRAIN_INPUT_DATA, 'wb'), train_inputs)
np.save(open(TRAIN_LABEL_DATA, 'wb'), train_labels)

# 정제된 텍스트를 csv 형태로 저장
clean_train_df.to_csv(TRAIN_CLEAN_DATA, index = False)


# 데이터 사전을 json 형태로 저장
json.dump(data_configs, open(DATA_CONFIGS, 'w'), ensure_ascii=False)

In [ ]:
test_data.head(10)

In [ ]:
clean_test_reviews = []

In [ ]:
for review in test_data['review']:
    clean_test_reviews.append(preprocessing(review, remove_stopwords = True))


clean_test_df = pd.DataFrame({'review': clean_test_reviews, 'id': test_data['id']})
test_id = np.array(test_data['id'])

text_sequences = tokenizer.texts_to_sequences(clean_test_reviews)
test_inputs = pad_sequences(text_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

In [ ]:
#토크나이저를 통해 인덱스 벡터로 만들 때 토크나이징 객체로 새롭게 만드는 것이 아니라
#기존에 학습 데이터에 적용한 토크나이저 객체를 사용하는 것이다.
#새롭게 만들 경우 학습 데이터와 평가 데이터 대한 각 단어들의 인덱스가 달라져서 모델에
#정상적으로 적용할 수 없기 때문이다.

In [ ]:
TEST_INPUT_DATA = 'test_input.npy'
TEST_CLEAN_DATA = 'test_clean.csv'
TEST_ID_DATA = 'test_id.npy'

np.save(open(TEST_INPUT_DATA, 'wb'), test_inputs)
np.save(open(TEST_ID_DATA, 'wb'), test_id)
clean_test_df.to_csv(TEST_CLEAN_DATA, index = False)

In [ ]:
#선형회귀모델

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [ ]:
train_data_using = pd.read_csv('train_clean.csv',header = 0, delimiter='\t',quoting=3)

reviews = list(train_data['review'])
sentiments = list(train_data['sentiment'])

In [ ]:
RANDOM_SEED = 42
TEST_SPLIT = 0.2

In [ ]:
vectorizer = TfidfVectorizer(min_df = 0.0, analyzer="char", sublinear_tf=True, ngram_range=(1,3), max_features=5000) 

X = vectorizer.fit_transform(reviews)
y = np.array(sentiments)

In [ ]:

X


In [ ]:
features = vectorizer.get_feature_names()
X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=TEST_SPLIT, random_state=RANDOM_SEED)

In [ ]:
lgs = LogisticRegression(class_weight='balanced') 
model = lgs.fit(X_train, y_train)

In [ ]:

predicted = lgs.predict(X_eval)

In [ ]:
predicted

In [ ]:
print("Accuracy: %f" % lgs.score(X_eval, y_eval))

In [ ]:
!pip install joblib

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
import pickle

import joblib

saved_model = pickle.dumps(lgs)

clf_from_pickle = pickle.loads(saved_model)

# Use the loaded pickled model to make predictions
clf_from_pickle.predict(X_eval)

print("Accuracy: %f" % clf_from_pickle.score(X_eval, y_eval))


In [ ]:
joblib.dump(lgs, 'filename.pkl') 
clf_from_joblib = joblib.load('filename.pkl') 
clf_from_joblib.predict(X)


In [ ]:
TEST_CLEAN_DATA = 'test_clean.csv'

test_data = pd.read_csv(TEST_CLEAN_DATA)

In [ ]:

testDataVecs = vectorizer.transform(test_data['review'])

In [ ]:
test_predicted = lgs.predict(testDataVecs)
print(test_predicted)

In [ ]:
test_data.head()

In [ ]:
answer_dataset = pd.DataFrame({'id': test_data['id'], 'sentiment': test_predicted})
answer_dataset.to_csv('lgs_tfidf_answer.csv', index=False, quoting=3)

In [ ]:
#내 데이터 불러오기

In [ ]:
my_data = pd.read_csv('../input/mysettt/myResults.csv', header = 0,delimiter=",")
my_data.head()


In [ ]:
clean_my_reviews = []
for review in my_data['review']:
    clean_my_reviews.append(preprocessing(review, remove_stopwords =True))
clean_my_df = pd.DataFrame({'review': clean_my_reviews,'id': my_data['id']})
my_id = np.array(my_data['id'])

tokenizer.fit_on_texts(clean_my_reviews)
text_sequences2 = tokenizer.texts_to_sequences(clean_my_reviews)
my_inputs = pad_sequences(text_sequences2,maxlen=MAX_SEQUENCE_LENGTH,padding='post')

In [ ]:
np.save(open("my_input.npy","wb"),my_inputs)
np.save(open("my_id.npy","wb"),my_id)
clean_my_df.to_csv("my_clean.csv",index=False)


In [ ]:
my_data2 = pd.read_csv("./my_clean.csv",header = 0,delimiter=",")

myDataVecs = vectorizer.transform(my_data2['review'].values.astype('U'))

my_predicted = lgs.predict(myDataVecs)
print(my_predicted)

In [ ]:
texts = list(my_data2['review'])
res = pd.DataFrame({'review':texts,'sentiment':my_predicted})
res.to_csv("check_this_one.csv")